MODEL LSTM - PREDICCIÓ PREUS

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# CONFIGURACIÓ GENERAL

# Directori amb els CSV preprocessats
BASE_PATH    = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\Conjunt de dades Preprocessades\Datasets"
# Directori amb els resultats del LSTM
RESULTS_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM"
# Llista d'arxius a processar
DATASETS     = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv"
]

# Paràmetres per a seqüències i divisió de dades
N_STEPS = 30
FEATURE_COLUMNS = [
    "Open", "High", "Low", "Volume",
    "EMA_7", "EMA_40", "MACD", "Signal_Line",
    "MACD_Hist", "RSI", "ATR"
]
TARGET_COLUMN = "Close"
TEST_RATIO = 0.10  # 10% per a test
VAL_RATIO  = 0.10  # 10% per a validació
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO  # restant per a entrenament

# FUNCIONS AUXILIARS

#Genera seqüències de longitud n_steps de X i y.
def create_sequences(X, y, n_steps=30):

    Xs, ys = [], []
    for i in range(n_steps, len(X)):
        Xs.append(X[i - n_steps:i])  # finestra de n_steps enrere
        ys.append(y[i])              # valor objectiu corresponent
    return np.array(Xs), np.array(ys)

#Reconstrueix el model LSTM amb els hiperparàmetres indicats i el compila amb pèrdua Huber i optimitzador Adam.
def build_lstm_model(sequence_length, n_features, units, n_layers, dropout, learning_rate):
 
    model = Sequential()
    for i in range(n_layers):
        return_seq = (i < n_layers - 1)
        if i == 0:
            # primera capa LSTM amb input_shape definit
            model.add(LSTM(units, return_sequences=return_seq,
                           input_shape=(sequence_length, n_features)))
        else:
            # capes següents
            model.add(LSTM(units, return_sequences=return_seq))
        model.add(Dropout(dropout))  # evitar sobreajust
    # capa de sortida lineal per regressió
    model.add(Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='huber', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

# Calcula MAE, RMSE i R2 sobre dades escalades.
def compute_metrics(model, X_scaled, y_scaled, scaler_y):
   
    y_pred_scaled = model.predict(X_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2, y_true, y_pred

# Recalcula indicadors tècnics (EMA, MACD, RSI, ATR) per al DataFrame.
def recompute_indicators(df):
  
    close = df['Close']
    df['EMA_7'] = close.ewm(span=7, adjust=False).mean()
    df['EMA_40'] = close.ewm(span=40, adjust=False).mean()
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    df['MACD'] = macd
    df['Signal_Line'] = signal
    df['MACD_Hist'] = macd - signal
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['ATR'] = (df['High'] - df['Low']).rolling(window=14).mean()

# BUCLE PRINCIPAL

for filename in DATASETS:
    dataset_name = os.path.splitext(filename)[0]
    print(f"\n===== Processant: {dataset_name} =====")

    # definim rutes de fitxers i directoris
    data_path    = os.path.join(BASE_PATH, filename)
    model_folder = os.path.join(RESULTS_PATH, dataset_name)
    weights_path = os.path.join(model_folder, f"{dataset_name}_best_weights.weights.h5")
    params_path  = os.path.join(model_folder, f"{dataset_name}_best_params.json")

    # comprovem si existeixen els fitxers necessaris
    if not os.path.isfile(weights_path) or not os.path.isfile(params_path):
        print(f"  ⚠️ Falten pesos o paràmetres per a {dataset_name}, s'omet.")
        continue

    # càrrega i neteja del CSV
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # creació de seqüències i divisió en train/val/test
    data_X = df[FEATURE_COLUMNS].values
    data_y = df[TARGET_COLUMN].values.reshape(-1,1)
    X_seq, y_seq = create_sequences(data_X, data_y, n_steps=N_STEPS)
    n_total = len(X_seq)
    train_end = int(n_total * TRAIN_RATIO)
    val_end   = train_end + int(n_total * VAL_RATIO)
    X_train, y_train = X_seq[:train_end], y_seq[:train_end]
    X_val, y_val     = X_seq[train_end:val_end], y_seq[train_end:val_end]
    X_test, y_test   = X_seq[val_end:], y_seq[val_end:]

    # construcció dels escaladors amb train+val
    scaler_X = MinMaxScaler().fit(X_seq[:val_end].reshape(-1,len(FEATURE_COLUMNS)))
    scaler_y = MinMaxScaler().fit(y_seq[:val_end])
    def scale_X(arr): return scaler_X.transform(arr.reshape(-1,len(FEATURE_COLUMNS))).reshape(arr.shape)
    X_train_s, X_val_s, X_test_s = scale_X(X_train), scale_X(X_val), scale_X(X_test)
    y_train_s, y_val_s, y_test_s = scaler_y.transform(y_train), scaler_y.transform(y_val), scaler_y.transform(y_test)

    # càrrega paràmetres i reconstrucció del model
    params = json.load(open(params_path,'r'))
    model = build_lstm_model(N_STEPS, len(FEATURE_COLUMNS), params['units'], params['n_layers'], params['dropout'], params['learning_rate'])
    model.load_weights(weights_path)
    print(f"  ✓ Model LSTM carregat amb pesos i paràmetres.")

    # avaluació del model en test i emmagatzematge de mètriques
    mae, rmse, r2, y_true, y_pred = compute_metrics(model, X_test_s, y_test_s, scaler_y)
    print(f"  → Test MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")
    pd.DataFrame({"Dataset":[dataset_name],"MAE":[mae],"RMSE":[rmse],"R2":[r2]}).to_csv(os.path.join(model_folder,f"{dataset_name}_metrics_test.csv"), index=False)

    # gràfic Real vs Predicció (Test)
    start_idx = N_STEPS + val_end
    dates_test = df['Date'].iloc[start_idx:start_idx+len(y_true)].reset_index(drop=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_test, y=y_true.flatten(), mode='lines', name='Real (Close)'))
    fig.add_trace(go.Scatter(x=dates_test, y=y_pred.flatten(), mode='lines', name='Predicció LSTM', line=dict(dash='dash')))
    fig.update_layout(title=f"{dataset_name} – Real vs Predicció (Test)", xaxis_title='Data', yaxis_title='Preu (Close)', template='plotly_white', xaxis_rangeslider_visible=True)
    fig.write_html(os.path.join(model_folder,f"{dataset_name}_test_plot.html"))
    print(f"  ✓ Gràfic Test desat.")

    # predicció autoregressiva per als propers 10 dies hàbils
    df_future = df.copy().reset_index(drop=True)
    recompute_indicators(df_future)
    last_seq = X_test_s[-1].reshape(1,N_STEPS,len(FEATURE_COLUMNS))
    future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1]+pd.Timedelta(days=1), periods=10)
    future_preds = []
    for date in future_dates:
        y_s = model.predict(last_seq, verbose=0)[0][0]
        y_r = scaler_y.inverse_transform([[y_s]])[0][0]
        future_preds.append(y_r)
        prev = df_future.iloc[-1]
        new = prev.copy()
        new.update({'Date':date,'Open':prev['Close'],'High':y_r,'Low':y_r,'Close':y_r})
        df_future = df_future.append(new,ignore_index=True)
        recompute_indicators(df_future)
        last_seq = scaler_X.transform(df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values).reshape(1,N_STEPS,len(FEATURE_COLUMNS))
    # desar prediccions futures
    pd.DataFrame({'Date':future_dates,'Predicted_Close':future_preds}).to_csv(os.path.join(model_folder,f"{dataset_name}_future_10days.csv"), index=False)
    print(f"  ✓ Prediccions futures desades.")



===== Procesando: Amazon_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, units=512, dropout=0.1, lr=0.001


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_best_weights.weights.h5
  → Test MAE=4.5752, RMSE=5.7316, R²=0.9283
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_plot.html

===== Procesando: Euro_Stoxx_50_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, uni

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  → Test MAE=51.0807, RMSE=64.9371, R²=0.7725
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_plot.html

===== Procesando: Google_Stock_Price_output =====
  ✓ Parámetros cargados: layers=1, units=512, dropout=0.5, lr=0.001


...

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_best_weights.weights.h5
  → Test MAE=2.1416, RMSE=2.6764, R²=0.6456
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\P&G_Stock_Price_output\P&G_Stock_Price_output_future_plot.html

===== Procesando: S&P500_Stock_Price_output =====
  ✓ Parámetros cargados: layers=2, units=128, dropout=0.1, lr=0.001


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 



  ✓ Pesos cargados desde: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_best_weights.weights.h5
  → Test MAE=71.1080, RMSE=90.2894, R²=0.8303
  ✓ Métricas guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_metrics_test.csv
  ✓ Gráfica Test guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_test_plot.html
  ✓ Predicciones futuras guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_10days.csv
  ✓ Gráfica futura guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_LSTM\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_plot.html


: 

MODEL HÍBRID - PREDICCIÓ PREUS

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
import xgboost as xgb

# CONFIGURACIÓ DIRECTORIS
BASE_DIR = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock"
DATA_DIR = os.path.join(BASE_DIR, "Conjunt de dades Preprocessades", "Datasets")
LSTM_WEIGHTS_DIR = os.path.join(BASE_DIR, "LSTM", "RESULTATS_LSTM")
OUTPUT_DIR = os.path.join(BASE_DIR, "LSTM", "RESULTATS_HIBRIDS")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Llista de fitxers de dades per processar
DATASETS = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv",
]

# Hiperparàmetres del model
N_STEPS = 30  # Nombre de passos temporal per a la seqüència entrada
FEATURE_COLUMNS = [  # Columnes d'entrada (característiques tècniques i mercat)
    "Open", "High", "Low", "Volume",
    "EMA_7", "EMA_40", "MACD", "Signal_Line",
    "MACD_Hist", "RSI", "ATR"
]
TARGET_COLUMN = "Close"  # Columna objectiu (preu de tancament)
TEST_RATIO = 0.10  # Percentatge de dades per test\NVAL_RATIO = 0.10  # Percentatge de dades per validació
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO  # Percentatge de dades per entrenament

# FUNCIONS AUXILIARS 

def load_dataset(path):
    df = pd.read_csv(path, parse_dates=["Date"]).sort_values("Date")
    return df.reset_index(drop=True)

# calcula Indicadors tècnics
def compute_technical_indicators(df):
   
    close = df[TARGET_COLUMN]
    # Exponential Moving Averages
    df["EMA_7"] = close.ewm(span=7, adjust=False).mean()
    df["EMA_40"] = close.ewm(span=40, adjust=False).mean()
    # MACD i línia de senyal
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    df["MACD"] = macd
    df["Signal_Line"] = signal
    df["MACD_Hist"] = macd - signal
    # RSI (Relative Strength Index)
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-8)
    df["RSI"] = 100 - (100 / (1 + rs))
    # ATR (Average True Range)
    df["ATR"] = (df["High"] - df["Low"]).rolling(window=14).mean()
    return df

# Genera seqüències temporals per a l'entrada del LSTM.
def create_sequences(data, n_steps=N_STEPS):
   
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-1])
        y.append(data[i, -1])
    return np.array(X), np.array(y)

#Construeix i compila el model LSTM amb els paràmetres donats.
def build_and_compile_lstm(input_shape, params):

    model = Sequential()
    for i in range(params['n_layers']):
        # Si no és l'última capa, retornar seqüència
        return_seq = i < params['n_layers'] - 1
        layer_args = dict(units=params['units'], return_sequences=return_seq)
        if i == 0:
            model.add(LSTM(**layer_args, input_shape=input_shape))
        else:
            model.add(LSTM(**layer_args))
        model.add(Dropout(params['dropout']))
    # Capa d'activació lineal per a regressió
    model.add(Dense(1, activation="linear"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(loss="huber", optimizer=optimizer, metrics=["mae"])
    return model

# Avalua el model: desescala prediccions i calcula MAE, RMSE i R2.
def evaluate_model(model, X, y, scaler):
    
    y_pred = model.predict(X, verbose=0)
    y_true = scaler.inverse_transform(y.reshape(-1, 1)).flatten()
    y_pred_real = scaler.inverse_transform(y_pred).flatten()
    return {
        'MAE': mean_absolute_error(y_true, y_pred_real),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred_real)),
        'R2': r2_score(y_true, y_pred_real),
        'y_true': y_true,
        'y_pred': y_pred_real
    }

# Entrena un model XGBoost per aprendre els residus del LSTM.
# Es divideix en 80% entrenament i 20% validació.
def train_xgb_on_residuals(X_feats, residuals):
   
    np.random.seed(42)
    idx = np.arange(len(X_feats))
    np.random.shuffle(idx)
    split = int(len(idx) * 0.8)
    train_idx, val_idx = idx[:split], idx[split:]

    X_train_xgb = X_feats[train_idx]
    y_train_xgb = residuals[train_idx]
    X_val_xgb = X_feats[val_idx]
    y_val_xgb = residuals[val_idx]

    xgb_reg = xgb.XGBRegressor(
        n_estimators=1000,
        learning_rate=0.01,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    xgb_reg.fit(
        X_train_xgb, y_train_xgb,
        eval_set=[(X_val_xgb, y_val_xgb)],
        verbose=False
    )
    return xgb_reg

#  Genera una gràfica interactiva amb Plotly comparant valors reals, LSTM i híbrids.
def plot_results(dates, true_vals, lstm_preds, hybrid_preds, title, output_path):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=true_vals, mode='lines', name='Real'))
    fig.add_trace(go.Scatter(x=dates, y=lstm_preds, mode='lines', name='LSTM Prediction'))
    fig.add_trace(go.Scatter(x=dates, y=hybrid_preds, mode='lines', name='Hybrid Prediction'))
    fig.update_layout(
        title=title,
        xaxis_title='Date', yaxis_title='Close Price',
        template='plotly_dark', xaxis_rangeslider_visible=True
    )
    fig.write_html(output_path)


def main():
    
    for file in DATASETS:
        dataset_name = os.path.splitext(file)[0]
        print(f"Procesant {dataset_name}...")

        # Carpeta de sortida per al dataset actual
        dataset_output_dir = os.path.join(OUTPUT_DIR, dataset_name)
        os.makedirs(dataset_output_dir, exist_ok=True)

        # Carrega i preprocessa dades
        df = load_dataset(os.path.join(DATA_DIR, file))
        df = compute_technical_indicators(df).dropna().reset_index(drop=True)

        # Prepara seqüències i divideix en train/val/test
        arr = df[FEATURE_COLUMNS + [TARGET_COLUMN]].values
        X_all, y_all = create_sequences(arr)
        n_samples = len(X_all)
        train_end = int(n_samples * TRAIN_RATIO)
        val_end = train_end + int(n_samples * VAL_RATIO)

        X_train, y_train = X_all[:train_end], y_all[:train_end]
        X_val, y_val = X_all[train_end:val_end], y_all[train_end:val_end]
        X_test, y_test = X_all[val_end:], y_all[val_end:]

        # Escalat de dades amb MinMaxScaler
        scaler_X = MinMaxScaler().fit(X_all.reshape(-1, X_all.shape[-1]))
        scaler_y = MinMaxScaler().fit(y_all.reshape(-1, 1))
        X_train_scaled = scaler_X.transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_val_scaled = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
        X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
        y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1))
        y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1))
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

        # Carrega paràmetres i pesos del model LSTM prèviament entrenat
        param_file = os.path.join(LSTM_WEIGHTS_DIR, dataset_name, f"{dataset_name}_best_params.json")
        weights_file = os.path.join(LSTM_WEIGHTS_DIR, dataset_name, f"{dataset_name}_best_weights.weights.h5")
        with open(param_file) as pf:
            params = json.load(pf)
        lstm = build_and_compile_lstm((N_STEPS, len(FEATURE_COLUMNS)), params)
        lstm.load_weights(weights_file)

        # Avaluació del model LSTM en test set
        lstm_metrics = evaluate_model(lstm, X_test_scaled, y_test_scaled, scaler_y)
        print(f"LSTM Test → MAE: {lstm_metrics['MAE']:.4f}, RMSE: {lstm_metrics['RMSE']:.4f}, R2: {lstm_metrics['R2']:.4f}")

        # Entrena XGBoost sobre residus de train+val
        train_val_scaled = np.concatenate([X_train_scaled, X_val_scaled])
        y_train_val_scaled = np.concatenate([y_train_scaled, y_val_scaled])
        y_pred_train_val = lstm.predict(train_val_scaled)
        true_train_val = scaler_y.inverse_transform(y_train_val_scaled).flatten()
        pred_train_val = scaler_y.inverse_transform(y_pred_train_val).flatten()
        residuals = true_train_val - pred_train_val

        raw_arr = df[FEATURE_COLUMNS].values[N_STEPS:]
        xgb_feats = np.hstack([raw_arr[:len(residuals)], pred_train_val.reshape(-1, 1)])
        xgb_model = train_xgb_on_residuals(xgb_feats, residuals)

        # Test del model híbrid (LSTM + XGBoost)
        lstm_test_pred = scaler_y.inverse_transform(lstm.predict(X_test_scaled)).flatten()
        raw_test = df[FEATURE_COLUMNS].values[N_STEPS + val_end:]
        xgb_test_feats = np.hstack([raw_test, lstm_test_pred.reshape(-1, 1)])
        hybrid_preds = lstm_test_pred + xgb_model.predict(xgb_test_feats)
        true_test = df[TARGET_COLUMN].values[N_STEPS + val_end:]

        mae_h = mean_absolute_error(true_test, hybrid_preds)
        rmse_h = np.sqrt(mean_squared_error(true_test,hybrid_preds))
        r2_h = r2_score(true_test, hybrid_preds)
        print(f"Híbrid Test → MAE: {mae_h:.4f}, RMSE: {rmse_h:.4f}, R2: {r2_h:.4f}")

        # Desa mètriques en CSV per posterior anàlisi
        metrics_df = metrics_df = pd.DataFrame([{
            **{'Dataset': dataset_name},
            **{f"{k}_LSTM": v for k, v in lstm_metrics.items() if k in ['MAE','RMSE','R2']},
            **{'MAE_HYBRID': mae_h, 'RMSE_HYBRID': rmse_h, 'R2_HYBRID': r2_h}
        }])
        metrics_df.to_csv(os.path.join(dataset_output_dir, f"{dataset_name}_metrics.csv"), index=False)

        # Gràfica comparativa LSTM vs Híbrid en test
        start_test_idx = N_STEPS + val_end
        dates_test = df['Date'].iloc[start_test_idx:start_test_idx + len(true_test)].reset_index(drop=True)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=dates_test, y=true_test, mode='lines', name='Real (Close)'))
        fig.add_trace(go.Scatter(x=dates_test, y=lstm_test_pred, mode='lines', name='Pred LSTM', line=dict(dash='dash')))
        fig.add_trace(go.Scatter(x=dates_test, y=hybrid_preds, mode='lines', name='Pred HÍBRIDA', line=dict(dash='dot')))
        fig.update_layout(
            title=f"{dataset_name}– Real vs Prediccions (Test, LSTM i HÍBRIDA)",
            xaxis_title='Data', yaxis_title='Preu (Close)',
            template='plotly_white', xaxis_rangeslider_visible=True
        )
        hybrid_html = os.path.join(dataset_output_dir, f"{dataset_name}_test_plot_hybrid.html")
        fig.write_html(hybrid_html)
        print(f"  ✓ Gràfica Test HÍBRID guardada a: {hybrid_html}")

        # Predicció autoregressiva híbrida per a 10 dies futurs
        df_future = df.copy().reset_index(drop=True)
        recompute_indicators(df_future)  # Recalcula indicadors en format iteratiu

        last_sequence = X_test_scaled[-1].copy().reshape(1, N_STEPS, len(FEATURE_COLUMNS))
        future_preds_lstm = []
        future_preds_hybrid = []
        future_dates = pd.bdate_range(start=df_future['Date'].iloc[-1] + pd.Timedelta(days=1), periods=10)

        for date in future_dates:
            # Predicció LSTM escalada -> desescalada
            y_pred_scaled = lstm.predict(last_sequence, verbose=0)[0][0]
            y_pred_real = scaler_y.inverse_transform([[y_pred_scaled]])[0][0]
            future_preds_lstm.append(y_pred_real)

            # Afegeix registre temporal amb predicció
            prev = df_future.iloc[-1]
            new_row = {
                'Date':        date,
                'Open':        prev['Close'],
                'High':        y_pred_real,
                'Low':         y_pred_real,
                'Volume':      prev['Volume'],
                'Close':       y_pred_real,
                'EMA_7':       np.nan,
                'EMA_40':      np.nan,
                'MACD':        np.nan,
                'Signal_Line': np.nan,
                'MACD_Hist':   np.nan,
                'RSI':         np.nan,
                'ATR':         np.nan
            }
            df_future.loc[len(df_future)] = new_row
            recompute_indicators(df_future)

            # Prepara nova seqüència per LSTM
            recent_features = df_future[FEATURE_COLUMNS].iloc[-N_STEPS:].values
            recent_scaled = scaler_X.transform(recent_features)
            last_sequence = recent_scaled.reshape(1, N_STEPS, len(FEATURE_COLUMNS))

            # Predicció XGBoost sobre residu per predicció híbrid
            X_last_future = df_future[FEATURE_COLUMNS].iloc[-1].values.reshape(1, -1)
            Xgb_feat = np.concatenate([X_last_future, np.array([[y_pred_real]])], axis=1)
            residual_xgb = xgb_model.predict(Xgb_feat)[0]
            y_pred_hybrid = y_pred_real + residual_xgb
            future_preds_hybrid.append(y_pred_hybrid)

            # Actualitza 'Close' iterativament per següent pas
            df_future.at[df_future.index[-1], 'Close'] = y_pred_hybrid

        # Desa prediccions futures en CSV per a anàlisi
        df_fut_pred = pd.DataFrame({
            "Date": future_dates,
            "Pred_Close_LSTM": future_preds_lstm,
            "Pred_Close_HÍBRID": future_preds_hybrid
        })
        fut_csv = os.path.join(dataset_output_dir, f"{dataset_name}_future_10days_hybrid.csv")
        df_fut_pred.to_csv(fut_csv, index=False)
        print(f"  ✓ Prediccions futures híbrides guardades a: {fut_csv}")

        # Figura històrica + futura des de Plotly
        fig_fut = go.Figure()
        fig_fut.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Històric Close'))
        fig_fut.add_trace(go.Scatter(x=future_dates, y=future_preds_hybrid, mode='lines+markers', name='Pred futura HÍBRIDA', line=dict(dash='dash')))
        fig_fut.update_layout(
            title=f"{dataset_name} – Predicció Pròxims 10 Dies (HÍBRID: LSTM + XGBOOST)",
            xaxis_title='Data', yaxis_title='Preu (Close)',
            template='plotly_white', xaxis_rangeslider_visible=True
        )
        fut_html = os.path.join(dataset_output_dir, f"{dataset_name}_future_plot_hybrid.html")
        fig_fut.write_html(fut_html)
        print(f"  ✓ Gràfica futura híbrida guardada a: {fut_html}")


if __name__ == '__main__':
    main()


Processing Amazon_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 3.6188, RMSE: 4.6093, R2: 0.9534
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Hybrid Test → MAE: 3.2784, RMSE: 4.2841, R2: 0.9597
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Amazon_Stock_Price_output\Amazon_Stock_Price_output_future_plot_hybrid.html
Processing Euro_Stoxx_50_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 68.0060, RMSE: 85.1416, R2: 0.5995
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Hybrid Test → MAE: 25.9500, RMSE: 32.4365, R2: 0.9419
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Euro_Stoxx_50_Stock_Price_output\Euro_Stoxx_50_Stock_Price_output_future_plot_hybrid.html
Processing Google_Stock_Price_output...


...

LSTM Test → MAE: 233.7392, RMSE: 295.7193, R2: 0.7778
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Hybrid Test → MAE: 171.6424, RMSE: 226.0883, R2: 0.8701
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\Indra_Stock_Price_output\Indra_Stock_Price_output_future_plot_hybrid.html
Processing P&G_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 



LSTM Test → MAE: 1.8727, RMSE: 2.3827, R2: 0.7218
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Hybrid Test → MAE: 1.1756, RMSE: 1.5143, R2: 0.8876
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\P&G_Stock_Price_output\P&G_Stock_Price_output_future_plot_hybrid.html
Processing S&P500_Stock_Price_output...


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 



LSTM Test → MAE: 81.7329, RMSE: 101.3684, R2: 0.7772
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Hybrid Test → MAE: 73.3193, RMSE: 86.9410, R2: 0.8361
  ✓ Gràfica Test HÍBRID guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_test_plot_hybrid.html
  ✓ Prediccions futures híbrides guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_10days_hybrid.csv
  ✓ Gràfica futura híbrida guardada a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\LSTM\RESULTATS_HIBRIDS\S&P500_Stock_Price_output\S&P500_Stock_Price_output_future_plot_hybrid.html
